In [1]:
import libcst as cst

In [2]:
code = ""
file_path = "tests/data/0.py"

with open(file_path, "r") as f:
    code = f.read()

In [3]:
class FunctionCollector(cst.CSTVisitor):
    def __init__(self):
        self.functions = []

    def visit_FunctionDef(self, node: cst.FunctionDef):
        self.functions.append(node)

In [4]:
tree = cst.parse_module(code)
collector = FunctionCollector()
tree.visit(collector)

Module(
    body=[
        ClassDef(
            name=Name(
                value='Solution',
                lpar=[],
                rpar=[],
            ),
            body=IndentedBlock(
                body=[
                    FunctionDef(
                        name=Name(
                            value='twoSum',
                            lpar=[],
                            rpar=[],
                        ),
                        params=Parameters(
                            params=[
                                Param(
                                    name=Name(
                                        value='self',
                                        lpar=[],
                                        rpar=[],
                                    ),
                                    annotation=None,
                                    equal=MaybeSentinel.DEFAULT,
                                    default=None,
                                    comma=Comma(

In [5]:
code = """
def x(a, b, /, c, *args, x, y, z, **kwargs):
    pass
"""

tree = cst.parse_module(code)
collector = FunctionCollector()
tree.visit(collector)

collector.functions[0].params

Parameters(
    params=[
        Param(
            name=Name(
                value='c',
                lpar=[],
                rpar=[],
            ),
            annotation=None,
            equal=MaybeSentinel.DEFAULT,
            default=None,
            comma=Comma(
                whitespace_before=SimpleWhitespace(
                    value='',
                ),
                whitespace_after=SimpleWhitespace(
                    value=' ',
                ),
            ),
            star='',
            whitespace_after_star=SimpleWhitespace(
                value='',
            ),
            whitespace_after_param=SimpleWhitespace(
                value='',
            ),
        ),
    ],
    star_arg=Param(
        name=Name(
            value='args',
            lpar=[],
            rpar=[],
        ),
        annotation=None,
        equal=MaybeSentinel.DEFAULT,
        default=None,
        comma=Comma(
            whitespace_before=SimpleWhitespace(
          

In [6]:
code = """
class Solution:
    def twoSum(self, a: int, *args,  b: str, c: dict, d: list[tuple[char, str]], e: list[[dict[str]]], **kwargs) -> List[int] | None:
        pass
"""


class PrintIndent(cst.CSTVisitor):
    def visit_IndentedBlock(self, node):
        # print(node)
        pass
    
    def visit_Param_annotation(self, node):
        # print(node)
        pass
        
    def visit_Parameters(self, node):
        # print(node)
        pass
    
    def visit_Parameters_posonly_params(self, node):
        # print(node)
        pass
    
    def visit_Param(self, node):
        # print(node)
        pass
    
    def visit_Subscript_slice(self, node):
        print(node)
        pass
    
    def visit_Slice(self, node):
        node
        return super().visit_Slice(node)


tree = cst.parse_module(code)
collector = FunctionCollector()
tree.visit(collector)

p = PrintIndent()
tree.visit(p)

res = ""

def subscript_dfs(current: cst.Subscript):
    res = ""
    
    if isinstance(current.value, cst.Name):
        res += current.value.value + "["
        
    if isinstance(current.value, cst.Attribute):
        res += current.value.attr.value + "["
        
    for s in current.slice:
        if isinstance(s.slice, cst.Index):
            if isinstance(s.slice.value, cst.Subscript):
                res += subscript_dfs(s.slice.value) + ", "
            
            elif isinstance(s.slice.value, cst.Name):
                res += s.slice.value.value + ", "
                
            elif isinstance(s.slice.value, cst.Attribute):
                res += s.slice.value.attr.value + ", "
            
            elif isinstance(s.slice.value, cst.SimpleString):
                res += s.slice.value.value
        
            else:
                res += f"[{str(type(s.value))}], "
                
    return res[:-2] + "]"

# subscript_dfs(collector.functions[0].params.params[2].annotation.annotation)

Subscript(
    value=Name(
        value='list',
        lpar=[],
        rpar=[],
    ),
    slice=[
        SubscriptElement(
            slice=Index(
                value=Subscript(
                    value=Name(
                        value='tuple',
                        lpar=[],
                        rpar=[],
                    ),
                    slice=[
                        SubscriptElement(
                            slice=Index(
                                value=Name(
                                    value='char',
                                    lpar=[],
                                    rpar=[],
                                ),
                                star=None,
                                whitespace_after_star=None,
                            ),
                            comma=Comma(
                                whitespace_before=SimpleWhitespace(
                                    value='',
                                ),


In [7]:
class AnnotationParser(cst.CSTVisitor):
    def __init__(self):
        self.result = ""
        self.subscript_stack = 0
    
    def visit_Comma(self, node):
        self.result += ", "
    
    def visit_Subscript(self, node):
        self.subscript_stack += 1
    
    def leave_Subscript(self, original_node):
        self.result += "]"
        
    def visit_Name(self, node):
        self.result += node.value
        if self.subscript_stack > 0:
            self.result += "["
            self.subscript_stack -= 1
            
    
    def visit_Attribute(self, node):
        self.result += node.attr.value
        if self.subscript_stack > 0:
            self.result += "["
            self.subscript_stack -= 1
    
    def visit_SimpleString(self, node):
        self.result += node.value
        if self.subscript_stack > 0:
            self.result += "["
            self.subscript_stack -= 1
            
class ParameterCollector(cst.CSTVisitor):
    def __init__(self):
        self.params: list[tuple[str, str]] = []
        
        # config
        self.allow_star_args = False
    
    def visit_Param(self, node: cst.Param):
        if node.name.value == "self":
            return False
        
        if not self.allow_star_args and (node.star == "*" or node.star == "**"):
            return False
        
        param_name = node.name.value
        
        if isinstance(node.star, str):
            param_name = node.star + param_name
        
        if not node.annotation:
            self.params.append((param_name, "[unknown]"))
            return False
        
        parser = AnnotationParser()
        node.annotation.visit(parser)
        self.params.append((param_name, parser.result))
        return False
    
collector = ParameterCollector()
tree.body[0].body.body[0].params.visit(collector)
collector.params

[('a', 'int'),
 ('b', 'str'),
 ('c', 'dict'),
 ('d', 'list[tuple[char, str]]'),
 ('e', 'list[dict[str]]')]

In [8]:
code = """
class Solution:
    def twoSum(self, a: int, *args,  b: str, c: dict, d: list[tuple[char, str]], e: list[[dict[str]]], **kwargs) -> List[int] | str | None :
        pass
"""

tree = cst.parse_module(code)
tree.body[0].body.body[0].returns.annotation.right

Name(
    value='None',
    lpar=[],
    rpar=[],
)

In [9]:
def twoSum(a: int, *args,  b: str, c: dict, d: list[tuple[char, str]], e: list[[dict[str]]], **kwargs) -> list[int] | str | None :
    pass

NameError: name 'char' is not defined

In [ ]:
class GoogleTemplate:
    @staticmethod
    def generate_docstring(**kwargs) -> str:
        """
        Args:
            indent (str): base indentation
            summary (str): a summary of class or function
            parameters (list[tuple[str, str]]): 
            returns (str)
        Returns
            str: the generated docstring
        """
        
        return GoogleTemplate._generate_section_summary("test")
        
        return ""
    
    @staticmethod
    def _generate_section_summary(summary: str) -> list[str]:
        return [summary]
    
    @staticmethod
    def _generate_section_args(args: list[tuple[str, str]]) -> list[str]:
        pass
    
    @staticmethod
    def _generate_section_references(references: str) -> list[str]:
        pass
    
    @staticmethod
    def _generate_section_notes(notes: list[str]) -> list[str]:
        pass
    
    @staticmethod
    def _generate_section_examples(args: list[str]) -> list[str]:
        pass
    
GoogleTemplate.generate_docstring()

['test']

In [11]:
code = """raise ValueError"""

tree = cst.parse_module(code)

tree.body

(SimpleStatementLine(
     body=[
         Raise(
             exc=Name(
                 value='ValueError',
                 lpar=[],
                 rpar=[],
             ),
             cause=None,
             whitespace_after_raise=SimpleWhitespace(
                 value=' ',
             ),
             semicolon=MaybeSentinel.DEFAULT,
         ),
     ],
     leading_lines=[],
     trailing_whitespace=TrailingWhitespace(
         whitespace=SimpleWhitespace(
             value='',
         ),
         comment=None,
         newline=Newline(
             value=None,
         ),
     ),
 ),)